<a href="https://colab.research.google.com/github/valerio-unifei/ECAA07/blob/main/ECAA07_05_Manipular.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL DML (*Data Manager Language*)

In [ ]:
%load_ext sql
%sql sqlite:///ecaa07_05_manipular.db

# Inserir Dados Novos na Tabela

```
INSERT INTO <Tabela> (<campo1>, <campo2>) VALUES (<valor1>, <valor2>)
```

In [ ]:
%%sql
CREATE TABLE IF NOT EXISTS Alunos (
  IdAluno INTEGER PRIMARY KEY,
  Aluno VARCHAR(255),
  Matricula INTEGER,
  Entrada DATETIME)

## Usando TODOS os campos da tabela

In [ ]:
%%sql
INSERT INTO Alunos
  VALUES (1,'Andrés Urdangarin Dorronsoro',201201001,'2012-09-27')

In [ ]:
%sql SELECT * FROM Alunos

In [ ]:
%sql INSERT INTO Alunos VALUES (2,'Anfilófio Neves',201201004,'2012-04-26')

In [ ]:
%sql INSERT INTO Alunos VALUES (2,'Anfilófio Neves',201201004,'2012-04-26')

## Determinando a chave primária incrementada

Última chave primária inserida (SQLite):

```
SELECT last_insert_rowid()
```



In [ ]:
%%sql
CREATE TABLE IF NOT EXISTS Dispositivos (
  IdDispositivo INTEGER PRIMARY KEY AUTOINCREMENT,
  Dispositivo VARCHAR(255) NOT NULL,
  TagID INTEGER,
  Criado DATETIME DEFAULT CURRENT_TIMESTAMP)

In [ ]:
%%sql
INSERT INTO Dispositivos
  (Dispositivo, TagID) VALUES ('Sensor Indutivo', 43252)

In [ ]:
%sql SELECT last_insert_rowid()

In [ ]:
%sql SELECT * FROM Dispositivos

# Inserindo Dados de Outras Tabelas

```
INSERT INTO <Tabela1> SELECT * FROM <Tabela2>
```

Criando uma tabela cópia de Dispositivos

In [ ]:
%%sql
CREATE TABLE IF NOT EXISTS Dispositivos2022 (
  IdDispositivo INTEGER PRIMARY KEY AUTOINCREMENT,
  Dispositivo VARCHAR(255) NOT NULL,
  TagID INTEGER,
  Unidade VARCHAR(30) DEFAULT '-',
  Criado DATETIME DEFAULT CURRENT_TIMESTAMP)

Copiando dados de dispositivos para dispositivos2022

In [ ]:
%%sql
INSERT INTO Dispositivos2022
  (Dispositivo, TagID, Criado)
  SELECT d.Dispositivo, d.TagID, d.Criado
    FROM Dispositivos AS d

Nova cópia

In [ ]:
%%sql
INSERT INTO Dispositivos2022
  (Dispositivo, TagID, Criado)
  SELECT d.Dispositivo, d.TagID, d.Criado
    FROM Dispositivos AS d

Exibindo apenas dados sem repetição (ignorando chave primária)

In [ ]:
%%sql
SELECT DISTINCT Dispositivo,TagID,Unidade,Criado
  FROM Dispositivos2022

# Atualizar Dados Existentes



```
UPDATE <Tabela> SET <campo>=<valor> WHERE <campo>=<filtro>
```



## Edição sem Filtro - todos os registros

Editando campo Unidade de dispositivos2022

In [ ]:
%sql UPDATE Dispositivos2022 SET Unidade = 'metros'

## Edição com Filtro (Where)

Inserindo novos valores

In [ ]:
%%sql
INSERT INTO Dispositivos2022
  (Dispositivo, TagID, Criado)
  SELECT d.Dispositivo, d.TagID, d.Criado
    FROM Dispositivos AS d

Editando apenas dados novos

In [ ]:
%%sql
UPDATE Dispositivos2022
  SET Unidade = 'litros'
  WHERE Unidade = '-'

## Edição com Cálculo

Edição com valores antigos modificados

In [ ]:
%%sql
UPDATE Dispositivos2022
  SET TagID = TagID - 200
  WHERE Unidade = 'metros'

In [ ]:
%sql SELECT * FROM Dispositivos2022

Atualizando data de criação



```
SET Criado = DATE('now')
```



In [ ]:
%%sql
UPDATE Dispositivos2022
  SET Criado = DATE('now')
  WHERE Unidade = 'litros'

In [ ]:
%sql SELECT * FROM Dispositivos2022

## Edição Simultânea

Edição de múltiplos valores simultaneamente



```
SET <campo1>=<valor1>, <campo2>=<valor2>,...
```



In [ ]:
%%sql
UPDATE Dispositivos2022
  SET TagID = TagID + 200, Criado = DATETIME(Criado, '+3 days')
  WHERE Unidade = 'litros'

In [ ]:
%sql SELECT * FROM Dispositivos2022

## Edição com Filtro Indireto (IN SELECT INNER JOIN)



```
WHERE <campo> IN (SELECT <campo> FROM <tabela>)

WHERE <campo> IN (
  SELECT <campo>
  FROM <tabela1> INNER JOIN <tabela2>
    ON <tabela1>.<campo> = <tabela2>.<campo>
  )

```



In [ ]:
%%sql
-- Proteção para nova execução
DROP TABLE IF EXISTS Maquinas;
DROP TABLE IF EXISTS Controles;
DROP TABLE IF EXISTS Sensores;
-- Tabela de máquinas da linha
CREATE TABLE Maquinas (
  Maquina_id INTEGER PRIMARY KEY AUTOINCREMENT,
  Maquina VARCHAR(255) NOT NULL,
  Criado DATETIME DEFAULT CURRENT_TIMESTAMP);
-- Tabela de controladores da linha
CREATE TABLE Controles (
  Controle_id INTEGER PRIMARY KEY AUTOINCREMENT,
  Controle VARCHAR(255) NOT NULL,
  Maquina_id INTEGER REFERENCES Maquinas(Maquina_id),
  Criado DATETIME DEFAULT CURRENT_TIMESTAMP);
-- Sensores
CREATE TABLE Sensores (
  Sensor_id INTEGER PRIMARY KEY AUTOINCREMENT,
  Sensor VARCHAR(255) NOT NULL,
  Tipo VARCHAR(10),
  Controle_id INTEGER REFERENCES Controles(Controle_id),
  Criado DATETIME DEFAULT CURRENT_TIMESTAMP);
-- Inserindo
INSERT INTO Maquinas(Maquina) VALUES ('Prensa 5032');
INSERT INTO Controles(Controle, Maquina_id) VALUES ('CLP 123',last_insert_rowid());
INSERT INTO Sensores(Sensor,Controle_id,Tipo) VALUES ('M43897',last_insert_rowid(),'Temp');
INSERT INTO Maquinas(Maquina) VALUES ('Cortadora 3241');
INSERT INTO Controles(Controle, Maquina_id) VALUES ('CLP 124',last_insert_rowid());
INSERT INTO Sensores(Sensor,Controle_id,Tipo) VALUES ('K438975',last_insert_rowid(),'Umid');
INSERT INTO Maquinas(Maquina) VALUES ('Solagem 7652');
INSERT INTO Controles(Controle, Maquina_id) VALUES ('CLP 125',last_insert_rowid());
INSERT INTO Sensores(Sensor,Controle_id,Tipo) VALUES ('V213432',last_insert_rowid(),'Tensao');
INSERT INTO Maquinas(Maquina) VALUES ('Elevador 6234');
INSERT INTO Controles(Controle, Maquina_id) VALUES ('CLP 126',last_insert_rowid());
INSERT INTO Sensores(Sensor,Controle_id,Tipo) VALUES ('V65730',last_insert_rowid(),'Tensao');

In [ ]:
%sql SELECT * FROM Maquinas;

In [ ]:
%sql SELECT * FROM Controles;

In [ ]:
%sql SELECT * FROM Sensores;

Atualizando as Máquinas pelo sensores de Tensão do sistema:

In [ ]:
%%sql
UPDATE Maquinas SET Maquina = Maquina || " - Atualizada"
  WHERE Maquina_id IN (
    SELECT c.Maquina_id FROM Controles as c
      INNER JOIN Sensores as s ON c.Controle_id = s.Controle_id
      WHERE s.Tipo = "Tensao"
  )

In [ ]:
%sql SELECT * FROM Maquinas;

# Remover Dados



```
DELETE FROM <Tabela WHERE <campo> = <filtro>
```



In [ ]:
%sql DELETE FROM Dispositivos2022 WHERE IdDispositivo = 19

In [ ]:
%sql SELECT * FROM Dispositivos2022

## Utilizando campo de remoção

In [ ]:
%%sql
ALTER TABLE Dispositivos2022
  ADD COLUMN Apagado DATETIME DEFAULT NULL

In [ ]:
%%sql
UPDATE Dispositivos2022 SET Apagado = DATETIME('now')
  WHERE IdDispositivo = 16

In [ ]:
%sql SELECT * FROM Dispositivos2022 WHERE Apagado IS NULL

In [ ]:
%sql SELECT * FROM Dispositivos2022

# SQL TCL (*Transaction Control Language*)

Gestão do fluxo de informação da DML
```
BEGIN TRANSACTION;

UPDATE ...
UPDATE ...
UPDATE ...
UPDATE ...
UPDATE ...
UPDATE ...

INSERT INTO ...
INSERT INTO ...
INSERT INTO ...
INSERT INTO ...
INSERT INTO ...
INSERT INTO ...

COMMIT;
```



```
BEGIN TRANSACTION;

...

ROLLBACK;
```



## Inserindo registros com envio individual

In [ ]:
%%time
import pandas as pd
import sqlite3

df = pd.read_csv('https://raw.githubusercontent.com/valerio-unifei/ECAA07/main/Bancos/tr2_8867.csv')
print(f'Tabela linhas:{df.shape[0]} colunas:{df.shape[1]}')

print('Abrindo/Criando banco')
cn = sqlite3.Connection('tran_exemplo.db')

print('Criando Tabela Medidas')
cn.execute('DROP TABLE IF EXISTS Medidas')
cn.execute('''CREATE TABLE Medidas (
  Medidas_id INTEGER PRIMARY KEY AUTOINCREMENT,
  Medida REAL,
  Criado DATETIME DEFAULT CURRENT_TIMESTAMP)
''')

print('Inserindo Registros')
for id, row in df.iterrows():
  cn.execute('INSERT INTO Medidas(Medida) VALUES (?)',[row[3]])
  cn.commit()

print('Fecha o banco')
cn.close()

## Inserindo via Transação

In [ ]:
%%time
import pandas as pd
import sqlite3

df = pd.read_csv('https://raw.githubusercontent.com/valerio-unifei/ECAA07/main/Bancos/tr2_8867.csv')
print(f'Tabela linhas:{df.shape[0]} colunas:{df.shape[1]}')

print('Abrindo/Criando banco')
cn = sqlite3.Connection('tran_exemplo.db')

print('Criando Tabela Medidas')
cn.execute('DROP TABLE IF EXISTS Medidas')
cn.execute('''CREATE TABLE Medidas (
  Medidas_id INTEGER PRIMARY KEY AUTOINCREMENT,
  Medida REAL,
  Criado DATETIME DEFAULT CURRENT_TIMESTAMP)
''')

print('Inserindo Registros')
for id, row in df.iterrows():
  cn.execute('INSERT INTO Medidas(Medida) VALUES (?)',[row[3]])
# Transação só ocorre 1 vez para todos os registros
cn.commit()

print('Fecha o banco')
cn.close()